# How to run the code

All code in this repository that has been developed for the final is contained in the module ```final.py```. All other code is separated into many other files and was developed for the midterm. 

To run the code for the final, open a python envrironment in the directory and import the module using ```import final.py```.


## Bibliography
We used the following resources to complete this project:

    * [os library documentation](https://github.com/iglesias-cardinale/CP1-24-final)